In [ ]:
async def get_earnings_by_company_id(self, company_id: str) -> List[EarningsMetadata]:
    """Fetch earnings documents from the database"""
    query = f"metadata_txt:{company_id}"
    headers = {
        "Cookie": f"GSSSO={self.gssso_token}",
        "responseType": "application/json"
    }
    base_url = "https://vip.gsam.prod.search.lex.site.gs.com/searchService/rest/search/bulk/query/table:com.gs.swm.genai"
    full_url = f"{base_url}%20AND%20({query})"
    
    result = []
    async with aiohttp.ClientSession() as session:
        retry = 0
        while retry < 5:
            try:
                async with session.get(full_url, headers=headers) as response:
                    if response.status != 200:
                        retry += 1
                        await asyncio.sleep(1)
                        continue
                        
                    content = await response.text()
                    soup = BeautifulSoup(content, 'lxml')
                    
                    # Process each lexDocument
                    for doc in soup.find_all("lexDocument"):
                        # First find metadata_txt field
                        metadata_field = doc.find('field', {'name': 'metadata_txt'})
                        if not metadata_field:
                            continue
                            
                        try:
                            # Get metadata JSON
                            metadata_value = metadata_field.find('values').text.strip()
                            metadata = json.loads(metadata_value)
                            
                            # Get objecturl if exists
                            objecturl_field = doc.find('field', {'name': 'objecturl'})
                            if objecturl_field:
                                metadata['objecturl'] = objecturl_field.find('values').text.strip()
                            
                            # Create earnings metadata object
                            earnings_metadata = EarningsMetadata(**metadata)
                            result.append(earnings_metadata)
                        except Exception as e:
                            print(f"Error parsing document: {e}")
                            continue
                    
                    # If we got here, we successfully processed the response
                    break
                    
            except Exception as e:
                print(f"Request error: {e}")
                retry += 1
                await asyncio.sleep(1)
                
    return result